While mode and for mode

In [75]:
import numpy as np


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [32]:
from typing import Annotated, Sequence

# It's just so appropriate here.
import numpy

from functools import singledispatch
class Instruction():
    pass

from dataclasses import dataclass

@dataclass
class ValueRange:
    min: float
    max: float


In [21]:
from dataclasses import dataclass
from typing import Tuple
from typing import Self
from typing import Callable


@dataclass
class Operation(Instruction):
    def __init__(self: Self,
                 function: Callable[..., float],
                 target: Annotated[int, ValueRange(0, float('inf'))],
                 args: Tuple[int, ...]):
        
        #: Function of the operation.
        self.function: Callable[..., float] = function

        #: Index of the target register.
        self.target: int = target
        
        has_no_register_operand: bool = True
        for reg in args:
            if reg >= 0:
                has_no_register_operand = False

        if (has_no_register_operand):
            raise ValueError(f"Operand registers are all constants")
        #: Indices of operand registers
        self.args: Tuple[int, ...] = args

    def __str__(self: Self):
        args: str = ', '.join((f"r[{x}]" if x >= 0 else f"c[{-x-1}]" for x in self.args))
        function_name: str = getattr(self.function,
                                     '__name__',
                                     repr(self.function))
        
        return f"r[{self.target}] <- {function_name}({args})"
    
    __repr__ = __str__


Use these parameters:
(a) arity

two modes:
if `inputs_can_change` is `True`: then append input registers to the end of the registers
otherwise, append input registers to constants

Append output registers to the last



In [113]:
# arity: number of input registers

# parameter: number of output registers

# reg_length: length of registers

# inputs_can_change:

from typing import Self

from numbers import Number

from typing import TypeVar, Generic
T = TypeVar("T")


from numpy.typing import NDArray

from numpy import float64

class LinearProgram():
    def __init__(self: Self,
                 coarity: int,
                 inputs: Sequence[float],
                 input_can_change: bool,
                 reg_length: int,
                 constants: Sequence[float],
                 initialiser: float | Callable[[], float]):
        
        #: Number of output registers
        self.coarity: int
        self.coarity = coarity

        self.registers: NDArray[np.float64]

        self.constants: NDArray[np.float64]

        if isinstance(initialiser, Number):
            self.registers = np.full(reg_length, initialiser, dtype=float64)
        elif (callable(initialiser)):
            self.registers = np.empty(reg_length, dtype=float64)
            for i in range(len(self.registers)):
                self.registers[i] = initialiser()
        else:
            raise ValueError("Initialiser is not callable and not a value.")
        
        # Initialise constants
        self.constants = np.fromiter(constants, dtype=float64)
        self.constants.flags.writeable = False

        # Append input registers to either constants or registers
        if input_can_change:
            self.registers = np.append(self.registers, inputs)
        else:
            self.constants = np.append(self.constants, inputs)

    def run(self: Self, instructions: Sequence[Instruction]) -> NDArray[np.float64]:
        for instruction in instructions:
            self.run_instruction(instruction)

        return self.get_output_values()

    def run_instruction(self: Self, instruction: Instruction) -> None:
        match instruction:
            case Operation():
                self._run_operation(instruction)

    def get_output_values(self: Self) -> NDArray[np.float64]:
        return self.registers[:self.coarity]


    def _run_operation(self: Self, instruction: Operation):
        if instruction.target < 0:
            raise ValueError(f"Malformed instruction: assignment to c[{-instruction.target-1}]")
        else:
            self.registers[instruction.target] =\
                instruction.function(
                    *(self.constants[-i-1] if i < 0 else self.registers[i]\
                      for i in instruction.args))

        print(str(instruction))

    def __str__(self: Self):
        return (f"Linear program. Current output values: {self.get_output_values()}\n") +\
                f"Constants c = {str(self.constants)}" +\
                f"Registers r = {str(self.registers)}"

    __repr__ = __str__

In [107]:
import random

A = LinearProgram(coarity = 3,
                  inputs = (1,2,3),
                  input_can_change = False,
                  reg_length = 4,
                  constants = (5,6,7),
                  initialiser = random.random)

In [109]:
def add(a,b):
    return a + b

def sub(a,b):
    return a + b


In [112]:
oprs = [Operation(add, 1, (2, 3)),
        Operation(sub, 0, (1, 2)),
        Operation(add, 2, (-2, 2)),]

A.run(oprs)
print(str(A))


r[1] <- add(r[2], r[3])
r[0] <- sub(r[1], r[2])
r[2] <- add(c[1], r[2])
Linear program. Current output values: [19.30583654 24.81907673  0.48675981]
Constants c = [5. 6. 7. 1. 2. 3.]Registers r = [38.12491327 19.30583654 24.81907673  0.48675981]


In [ ]:
from abc import ABC

type Runner = Callable[[LinearProgram, Sequence[Instruction]], None]

class StructToLines(ABC):
    def __init__(self: Self):
        self.action: Callable[[Sequence[Instruction]], None]

class StructUntilLabel(ABC):
    def __init__(self: Self):
        self.action: Callable[[Sequence[Instruction]], None]

class StructNextLine(ABC):
    def __init__(self: Self):
        self.action: Runner

    @property
    def action(self, lgp, ) -> None:
        Runner


    

# for, while, if

In [54]:
from functools import singledispatch

class A():
    pass
class B():
    pass
class C():
    pass

def returnA(o):
    return "A"

def returnB(o):
    return "B"

def returnC(o):
    return "C"

def dispatch_by_class(o):
    if isinstance(o, A):
        return returnA(o)
    elif isinstance(o, B):
        return returnB(o)
    else:
        return returnC(o)
    
def dispatch_by_match(o):
    
    match o:
        case A():
            return returnA(o)
        case B():
            return returnB(o)
        case C():
            return returnC(o)
        
    
def dispatch_with_dict(o):
    dicts = {
        A: returnA,
        B: returnB,
        C: returnC
    }
    return dicts[type(o)](o)

@singledispatch
def dispatch(o):
    raise TypeError("fail to dispatch")

@dispatch.register(A)
def dispatch_A(o):
    return "A"

@dispatch.register(B)
def dispatch_B(o):
    return "B"

@dispatch.register(C)
def dispatch_C(o):
    return "C"

from functools import singledispatchmethod

class PRINTER():

    @singledispatchmethod
    def print_me(self, other):
        raise TypeError("what")

    @print_me.register
    def _(self, other: A):
        return returnA(other)

    @print_me.register
    def _(self, other: B):
        return returnB(other)

    @print_me.register
    def _(self, other: C):
        return returnC(other)
    
P = PRINTER()


In [55]:
a = A()
b = B()
c = C()

In [56]:
%timeit dispatch_with_dict(a)

284 ns ± 29.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [57]:
%timeit dispatch_by_class(a)

109 ns ± 7.66 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [58]:
%timeit dispatch(a)

402 ns ± 4.06 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [59]:
%timeit P.print_me(a)

2.47 µs ± 69 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [60]:
%timeit dispatch_by_match(a)

176 ns ± 10.4 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
